# Add Fossil Classification for a given Holding File
## Prepare environment

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

# Classify quarterly holdings file

In [5]:
# TODO: add better matching for non-Israeli companies, using Figi as name normalization, fall back to fuzzy matching on normalized name and then fuzzy matching on original name

In [6]:
company_or_fund_level = "company"
year = "2023"
q = "3"
folder_path = "data/downloaded reports/" + company_or_fund_level + " reports/" + year + "Q" + q + "/"
holdings_path = folder_path + "holdings_for_cls.csv"

In [8]:
classify_holdings(holdings_path)


1. Preparing holding file

** Holdings file for classification **
data/downloaded reports/company reports/2023Q3/holdings_for_cls.csv
columns: Index(['שם המנפיק/שם נייר ערך', 'מספר ני"ע', 'מספר מנפיק', 'שווי',
       'שעור מנכסי אפיק ההשקעה', 'שעור מסך נכסי השקעה', 'holding_type',
       'זירת מסחר', 'תאריך רכישה', 'ערך נקוב', 'שער', 'שעור מערך נקוב מונפק',
       'ענף מסחר', 'SystemName', 'ParentCorpName', 'ReportPeriodDesc'],
      dtype='object')


/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:79: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  isin_cnt = sum(df[col].astype(str).str.strip().str.contains(isin_pattern, na=False))



Holding file ISIN col is: מספר ני"ע
number of ISINs: 6555 out of 25414 rows


/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pattern_cnt = sum(df[col].astype(str).str.strip().str.contains(pattern, na=False))



Holding file Israel Corp col is: מספר מנפיק
number of Israel Corp Numbers: 17813 out of 25414 rows

2. Preparing mapping files

3. Enriching holding file


...


is_fossil coverage before propagation by מספר ני"ע:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64

is_fossil coverage after propagation by מספר ני"ע:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64

Propagating by ISIN


/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_col_not_null['is_fossil'] = grouped_by_prop_col['is_fossil'].transform(
/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_col_not_null['is_fossil'] = grouped_by_prop_col['is_fossil'].transform(



is_fossil coverage before propagation by ISIN:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64

is_fossil coverage after propagation by ISIN:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64

Propagating by LEI

is_fossil coverage before propagation by LEI:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64

is_fossil coverage after propagation by LEI:
0.00    21976
1.00     3352
NaN        86
Name: is_fossil, dtype: int64


/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:669: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["is_fossil_conflict"] = df[is_fossil_cols].mean(axis=1).between(0, 1, inclusive=False)
/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:669: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df["is_fossil_conflict"] = df[is_fossil_cols].mean(axis=1).between(0, 1, inclusive=False)



Writing results to data/downloaded reports/company reports/2023Q3/holdings_for_cls with fossil classification.csv


# Manual Review
In a google spreadsheet or excel.
Download the fully classifed file into a csv, then use it in holding_cls_path to update prev_class (see below).

## Tips
1. Look at the output of the script, review conflicting classification (by ISIN, LEI, Israeli security number)
2. Look at holdings that get is_fossil_conflict=True
3. Sort by security name, Israeli security number or ISIN for faster manual classification
4. Carefully review holdings that have only is_fossil by FFF name match, as there are false matches.
<br>Review both holdings for which all the other is_fossil_x flags are null, and such that have is_fossil by FFF = 1and other is_fossil_x = 0
5. Review holdings from suspicious industries: energy, oil and gas, utilities, materials.

# Add classification results to prev_class

In [9]:
holdings_cls_path = folder_path + "holdings_for_cls with fossil classification - reviewed.csv"
prev_class_path = "data_sources/prev_class.csv"
update_prev_class(holdings_cls_path, prev_class_path)
# prev_class_fixed = add_all_id_types_to_holdings(prev_class, tlv_s2i, isin2lei)

Adding classifications to prev_class, saving the previous version as data_sources/prev_class backup/prev_class 2023-10-02 14-59-26.csv


# Classify fund holdings
## Israeli funds
Data is scraped from https://mayaapi.tase.co.il/api/fund/details?fundId=
<br>Page address: https://maya.tase.co.il/fund/5132287?view=assets

In [ ]:
import json

In [ ]:
response_directory = "data/holdings_for_classification/5132287/"
response_path = response_directory + "response.json"
fund = pd.read_json(response_path, orient="index")
assets = pd.DataFrame(fund.loc["AssetCompostion"][0]['Assets'])
# holdings["AssetCompostion"].head()
cols_rename = {
    'AssetName': 'שם המנפיק/שם נייר ערך',
    'IdentityCd': 'מספר ני"ע',
    'Id': 'fund_id'
}
assets = assets.rename(cols_rename, axis=1)
assets["מספר מנפיק"] = '00'
assets["מספר תאגיד"] = '00'
assets.to_csv(response_directory+"assets.csv", index=False)

In [ ]:
classify_holdings(response_directory+"assets.csv")

## holdings CSV file

In [ ]:
holdings_csv_dir = "/Users/urimarom/PycharmProjects/fossil_classification/data/holdings_for_classification/IE000PSF3A70/"
holdings_filename = 'fund_weights.csv'
holdings_csv_path = holdings_csv_dir + holdings_filename
holdings = pd.read_csv(holdings_csv_path)
cols_rename = {
    'Name': 'שם המנפיק/שם נייר ערך',
    'ISIN': 'מספר ני"ע',
    'Type of Security': 'holding_type'
}
holdings = holdings.rename(cols_rename, axis=1)
# fix missing columns
holdings["מספר מנפיק"] = '00'
holdings["מספר תאגיד"] = '00'
holdings["is_fossil_prev_il_sec_num"] = np.nan
holdings.to_csv(holdings_csv_dir+"fund_weights_fixed.csv", index=False)

In [ ]:
classify_holdings(holdings_csv_dir+"fund_weights_fixed.csv")